In [ ]:
!nvidia-smi

Fri Dec  8 03:58:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!pip install --quiet transformers
!pip install --quiet pytorch-lightning==1.1.1
!pip install --quiet tokenizers
!pip install --quiet sentencepiece
!pip install termcolor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.7/669.7 kB 9.3 MB/s eta 0:00:00


In [2]:
!pip install --upgrade pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.9/800.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.1.1
    Uninstalling pytorch-lightning-1.1.1:
      Successfully uninstalled pytorch-lightning-1.1.1


In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import argparse
import glob
import os

import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from termcolor import colored
import textwrap
from keras.callbacks import ModelCheckpoint


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [5]:
pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Seed set to 42


42

In [6]:
def extract_questions_and_answers(factoid_path: Path):
    """
    Extracts questions and their corresponding answers from a factoid JSON file.

    Parameters:
        factoid_path (Path): Path to the factoid JSON file.

    Returns:
        DataFrame: Pandas DataFrame containing extracted data with columns:
                   question, context, answer_text, answer_start, answer_end.
    """
    # Open the JSON file
    with factoid_path.open() as json_file:
        data = json.load(json_file)

    # Extract paragraphs containing questions
    questions = data["data"][0]["paragraphs"]

    data_rows = []

    # Iterate through each paragraph
    for question in questions:
        context = question["context"]
        # Iterate through each question and its answers
        for question_and_answers in question["qas"]:
            question_text = question_and_answers["question"]
            answers = question_and_answers["answers"]
            # Iterate through each answer
            for answer in answers:
                answer_text = answer["text"]
                answer_start = answer["answer_start"]
                answer_end = answer_start + len(answer_text)

                # Append extracted data to the list
                data_rows.append({
                    "question": question_text,
                    "context": context,
                    "answer_text": answer_text,
                    "answer_start": answer_start,
                    "answer_end": answer_end
                })

    # Convert the list of dictionaries to a Pandas DataFrame
    return pd.DataFrame(data_rows)

In [7]:
from pathlib import Path

# List of paths to factoid JSON files
factoid_paths = [sorted(list(Path("/kaggle/input/bioasq/BioASQ/").glob("BioASQ-train-*")))]

def filter_paths(paths):
    """
    Filter paths based on specific substrings.

    Parameters:
        paths (list of Path): List of paths to be filtered.

    Returns:
        list of Path: Filtered list of paths containing substrings '4b', '5b', or '6b'.
    """
    filtered_paths = [path for path in paths if '4b' in str(path) or '5b' in str(path) or '6b' in str(path)]
    return filtered_paths

# Filter factoid paths
filtered_paths = filter_paths(factoid_paths[0])

[PosixPath('/content/drive/MyDrive/Dataset/BioASQ-train-factoid-4b.json'),
 PosixPath('/content/drive/MyDrive/Dataset/BioASQ-train-factoid-5b.json')]

In [8]:
# List to store DataFrames extracted from factoid JSON files
dfs = []

# Iterate through each filtered factoid path
for factoid_path in filtered_paths:
    # Extract questions and answers from the factoid JSON file and append the resulting DataFrame
    dfs.append(extract_questions_and_answers(factoid_path))

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dfs)

In [9]:
df.head()

,question,context,answer_text,answer_start,answer_end
0,What is the inheritance pattern of Li–Fraumeni...,Balanced t(11;15)(q23;q15) in a TP53+/+ breast...,autosomal dominant,213,231
1,What is the inheritance pattern of Li–Fraumeni...,Genetic modeling of Li-Fraumeni syndrome in ze...,autosomal dominant,105,123
2,Which type of lung cancer is afatinib used for?,Clinical perspective of afatinib in non-small ...,EGFR-mutant NSCLC,1203,1220
3,Which hormone abnormalities are characteristic...,"DOCA sensitive pendrin expression in kidney, h...",thyroid,419,426
4,Which hormone abnormalities are characteristic...,Clinical and molecular characteristics of Pend...,thyroid,705,712


In [10]:
df = df.drop_duplicates(subset = ["question"]).reset_index(drop = True)

In [11]:
# Path to the pre-trained T5 model if you are using Google Colab
MODEL_NAME = "t5-base"

In [12]:
# Initialize a T5 tokenizer using a pre-trained model
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [13]:
class BioQADataset(Dataset):
    """
    Dataset class for BioASQ data to be used with T5 model.

    Parameters:
        data (pd.DataFrame): DataFrame containing the data.
        tokenizer (T5Tokenizer): Tokenizer for encoding text data.
        source_max_token_len (int): Maximum length for source encoding.
        target_max_token_len (int): Maximum length for target encoding.
    """

    def __init__(
        self,
        data: pd.DataFrame,
        tokenizer: T5Tokenizer,
        source_max_token_len: int = 396,
        target_max_token_len: int = 32
    ):
        """
        Initializes the BioQADataset.

        Args:
            data (pd.DataFrame): DataFrame containing the data.
            tokenizer (T5Tokenizer): Tokenizer for encoding text data.
            source_max_token_len (int, optional): Maximum length for source encoding. Defaults to 396.
            target_max_token_len (int, optional): Maximum length for target encoding. Defaults to 32.
        """
        self.tokenizer = tokenizer
        self.data = data
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def __len__(self):
        """
        Returns the number of samples in the dataset.
        """
        return len(self.data)

    def __getitem__(self, index: int):
        """
        Gets a sample from the dataset at the specified index.

        Args:
            index (int): Index of the sample to retrieve.

        Returns:
            dict: A dictionary containing the encoded inputs and labels.
        """
        data_row = self.data.iloc[index]

        # Encode source (question and context)
        source_encoding = self.tokenizer(
            data_row["question"],
            data_row["context"],
            max_length=self.source_max_token_len,
            padding="max_length",
            truncation="only_second",
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        # Encode target (answer_text)
        target_encoding = self.tokenizer(
            data_row["answer_text"],
            max_length=self.target_max_token_len,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors="pt"
        )

        # Prepare labels (setting padding tokens to -100)
        labels = target_encoding["input_ids"]
        labels[labels == 0] = -100

        return {
            "question": data_row["question"],
            "context": data_row["context"],
            "answer_text": data_row["answer_text"],
            "input_ids": source_encoding["input_ids"].flatten(),
            "attention_mask": source_encoding["attention_mask"].flatten(),
            "labels": labels.flatten()
        }

In [14]:
# Create a sample dataset using BioQADataset class
sample_dataset = BioQADataset(df, tokenizer)

In [15]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.05)

In [16]:
class BioQADataModule(pl.LightningDataModule):
    """
    LightningDataModule for preparing data for training and testing a T5 model on BioASQ data.

    Parameters:
        train_df (pd.DataFrame): DataFrame containing training data.
        test_df (pd.DataFrame): DataFrame containing test/validation data.
        tokenizer (T5Tokenizer): Tokenizer for encoding text data.
        batch_size (int): Batch size for data loaders. Defaults to 8.
        source_max_token_len (int): Maximum length for source encoding. Defaults to 396.
        target_max_token_len (int): Maximum length for target encoding. Defaults to 32.
    """

    def __init__(
        self,
        train_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: T5Tokenizer,
        batch_size: int = 8,
        source_max_token_len: int = 396,
        target_max_token_len: int = 32
    ):
        """
        Initializes the BioQADataModule.

        Args:
            train_df (pd.DataFrame): DataFrame containing training data.
            test_df (pd.DataFrame): DataFrame containing test/validation data.
            tokenizer (T5Tokenizer): Tokenizer for encoding text data.
            batch_size (int, optional): Batch size for data loaders. Defaults to 8.
            source_max_token_len (int, optional): Maximum length for source encoding. Defaults to 396.
            target_max_token_len (int, optional): Maximum length for target encoding. Defaults to 32.
        """
        super().__init__()
        self.batch_size = batch_size
        self.train_df = train_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.source_max_token_len = source_max_token_len
        self.target_max_token_len = target_max_token_len

    def setup(self, stage=None):
        """
        Setup datasets for training and testing.

        Args:
            stage (str): Stage of training, either 'fit' or 'test'. Defaults to None.
        """
        self.train_dataset = BioQADataset(
            self.train_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len
        )

        self.test_dataset = BioQADataset(
            self.train_df,
            self.tokenizer,
            self.source_max_token_len,
            self.target_max_token_len
        )

    def train_dataloader(self):
        """
        Returns a DataLoader for the training dataset.
        """
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4
        )

    def val_dataloader(self):
        """
        Returns a DataLoader for the validation dataset.
        """
        return DataLoader(
            self.test_dataset,
            batch_size=1,
            num_workers=4
        )

    def test_dataloader(self):
        """
        Returns a DataLoader for the test dataset.
        """
        return DataLoader(
            self.test_dataset,
            batch_size=1,
            num_workers=4
        )

In [17]:
# Set batch size and number of epochs
BATCH_SIZE = 8
N_EPOCHS = 6
# Create an instance of BioQADataModule
data_module = BioQADataModule(train_df, val_df, tokenizer, batch_size = BATCH_SIZE)

# Setup datasets for training and validation
data_module.setup()

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict = True)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [22]:
class BioQAModel(pl.LightningModule):
    """
    LightningModule for fine-tuning a T5 model on BioASQ data.

    Attributes:
        model (T5ForConditionalGeneration): Pre-trained T5 model.
        validation_step_outputs (list): List to store validation step outputs.
    """

    def __init__(self):
        """
        Initializes the BioQAModel.
        """
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)
        self.validation_step_outputs = []

    def forward(self, input_ids, attention_mask, labels=None):
        """
        Forward pass of the model.

        Args:
            input_ids (tensor): Input tensor representing tokenized inputs.
            attention_mask (tensor): Tensor representing attention mask.
            labels (tensor, optional): Tensor representing labels. Defaults to None.

        Returns:
            tuple: Tuple containing loss and logits.
        """
        output = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        """
        Training step for the model.

        Args:
            batch (dict): Batch of training data.
            batch_idx (int): Index of the batch.

        Returns:
            tensor: Loss tensor.
        """
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        """
        Validation step for the model.

        Args:
            batch (dict): Batch of validation data.
            batch_idx (int): Index of the batch.

        Returns:
            tensor: Loss tensor.
        """
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        self.validation_step_outputs.append(loss)
        return loss

    def test_step(self, batch, batch_idx):
        """
        Test step for the model.

        Args:
            batch (dict): Batch of test data.
            batch_idx (int): Index of the batch.

        Returns:
            tensor: Loss tensor.
        """
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        loss, outputs = self(input_ids, attention_mask, labels)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        """
        Configure optimizer for the model.

        Returns:
            torch.optim.Optimizer: Optimizer.
        """
        return AdamW(self.parameters(), lr=0.0001)

    def on_validation_epoch_end(self):
        """
        Performs actions at the end of each validation epoch.
        """
        epoch_average = torch.stack(self.validation_step_outputs).mean()
        self.log("validation_epoch_average", epoch_average)
        self.validation_step_outputs.clear()  # free memory

In [ ]:
# Create an instance of the BioQAModel
model = BioQAModel()

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

# Define a ModelCheckpoint callback to save the best model
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

# Define a TensorBoardLogger for logging training metrics
logger = TensorBoardLogger("training-logs", name = "bio-qa")

# Initialize the Trainer
trainer = Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=N_EPOCHS,
    accelerator="gpu",  # Use "gpu" for single GPU, "ddp" for distributed training
    logger=logger
    )

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ./training-logs

In [18]:
df.shape

(326, 5)

In [ ]:
trainer.fit(model,data_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 3, global step 912: 'val_loss' reached 0.05516 (best 0.05516), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 4, global step 1140: 'val_loss' reached 0.03759 (best 0.03759), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1


Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 5, global step 1368: 'val_loss' reached 0.02386 (best 0.02386), saving model to '/content/checkpoints/best-checkpoint.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=6` reached.


In [23]:
trained_model = BioQAModel.load_from_checkpoint("/content/drive/MyDrive/Dataset/best-checkpoint.ckpt")
trained_model.freeze()

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [24]:
def generate_answer(question):
    """
    Generates an answer to a given question using a pre-trained model.

    Args:
        question (dict): A dictionary containing the question and its context.

    Returns:
        str: The generated answer to the question.
    """
    # Tokenize the question and context
    source_encoding = tokenizer(
        question["question"],
        question["context"],
        max_length=396,
        padding="max_length",
        truncation="only_second",
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt"
    )

    # Move the tokenized data to the device used by the trained model
    source_encoding = {key: value.to(trained_model.device) for key, value in source_encoding.items()}

    # Generate answer IDs
    generated_ids = trained_model.model.generate(
        input_ids=source_encoding["input_ids"],
        attention_mask=source_encoding["attention_mask"],
        num_beams=1,  # Set to 1 for deterministic output
        max_length=80,  # Maximum length of the generated answer
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True,
        use_cache=True
    )

    # Decode the generated answer IDs into text
    preds = [
        tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        for generated_id in generated_ids
    ]

    # Concatenate all generated answers
    return "".join(preds)

In [26]:
sample_question = df.iloc[301]
question = sample_question['question']
answer = sample_question['answer_text']

# Print the question and the expected answer
print("Question:", question)
print("Expected Answer:", answer)

# Generate an answer using the function `generate_answer`
generated_answer = generate_answer({"question": question, "context": sample_question['context']})
print("Generated Answer:", generated_answer)

Which tool is used for promoterome mining using CAGE data?
CAGEr


In [27]:
print(sample_question)
generated_answer = generate_answer(sample_question)
print(generated_answer)

question        Which tool is used for promoterome mining usin...
context         CAGEr: precise TSS data retrieval and high-res...
answer_text                                                 CAGEr
answer_start                                                    0
answer_end                                                      5
Name: 301, dtype: object


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


CAGEr


In [ ]:
import pandas as pd
factoid_paths2 = [sorted(list(Path("/content/drive/MyDrive/Dataset/").glob("BioASQ-train-*")))]
filtered_paths2 = [path for path in factoid_paths[0] if '6b' in str(path)]
filtered_paths2

dfs = []

for factoid_path2 in filtered_paths2:
    dfs.append(extract_questions_and_answers(factoid_path2))


df = pd.concat(dfs)
# Loop through each row in the DataFrame
for index, row in df.iterrows():
    question = row['question']
    answer = row['answer_text']

    print("Question:", question)
    print("Answer:", answer)
    sample_question=df.iloc[index]
    generated_answer = generate_answer(sample_question)
    print("Generated Answer:", generated_answer)

    print()  # Print an empty line for separation

Question: What is the inheritance pattern of Li–Fraumeni syndrome?
Answer: autosomal dominant
Generated Answer: autosomal dominant

Question: Which type of lung cancer is afatinib used for?
Answer: EGFR-mutant NSCLC
Generated Answer: EGFR-mutant NSCLC

Question: Which hormone abnormalities are characteristic to Pendred syndrome?
Answer: thyroid
Generated Answer: thyroid

Question: Are most driver gene mutations synonymous or non-synonymous?
Answer: non-synonymous
Generated Answer: non-synonymous

Question: What is the major adverse effect of adriamycin(doxorubicin)?
Answer: cardiotoxicity
Generated Answer: cardiotoxicity

Question: Which is the branch site consensus sequence in U12-dependent introns?
Answer: UUCCUUAAC
Generated Answer: UUCCUUAAC

Question: What is the name for anorexia in gymnasts?
Answer: Anorexia Athletica
Generated Answer: Anorexia Athletica

Question: Which is the gene most commonly mutated in Tay-Sachs disease?
Answer: HEXA
Generated Answer: HEXA

Question: Which 

In [28]:
print(f'Question is: {question}')
print(f'Answer is: {answer}')
print(f'Predicted answer is: {generated_answer}')

Question is: Which tool is used for promoterome mining using CAGE data?
Answer is: CAGEr
Predicted answer is: CAGEr


In [30]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

def test1():
    answer = [['celiac','disease']]
    generated_answer = ['celiac','disease','is','harmful']
    score = sentence_bleu(answer, generated_answer , weights=(0.5,0.5,0,0))
    print(score)

test1()

0.408248290463863


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [31]:
print(f"BLEU score is: {(sentence_bleu([answer.split()], generated_answer.split(),weights=(0.5,0.5)))}")
print([answer.split()])
print(generated_answer)

BLEU score is: 1.491668146240062e-154
[['CAGEr']]
CAGEr


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [32]:
import locale
print(locale.getpreferredencoding())

ANSI_X3.4-1968


In [33]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
#!pip install sentence_transformers scikit-learn

In [34]:
#define Jaccard Similarity function
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

a = answer.split()
b = generated_answer.split()
print(a)
print(b)
#find Jaccard Similarity between the two sets
score = jaccard(a, b)
print(score)

['CAGEr']
['CAGEr']
1.0


In [37]:
#define Jaccard Similarity function
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

#a = answer.split()
a = 'Cager'
b = 'CAGEr'
print(a)
print(b)
#find Jaccard Similarity between the two sets
score = jaccard(a, b)
print(score)

Cager
CAGEr
0.25
